In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 一、数据

In [ ]:
import pandas as pd
train = pd.read_csv("../input/nCoV_100k_train.labled.csv")
test = pd.read_csv("../input/nCov_10k_test.csv")

In [ ]:
train.head()

In [ ]:
columns=['id','date','user','content','image','vedio','target']
train.columns=columns
test.columns=columns[:-1]
train.drop(['id','date','user','image','vedio'],axis = 1,inplace = True)
train.head()

## 预处理

In [ ]:
train['target'].value_counts()

In [ ]:
train_1=train.loc[(train['target']=='-1')] 
train0=train.loc[(train['target']=='0')] 
train1=train.loc[(train['target']=='1')] 
train = pd.concat([train_1,train0,train1])
train['target'].value_counts()
train.head()

In [ ]:
train.reset_index(drop=True,inplace=True)
train.head()

In [ ]:
train=train.sample(frac=1)
train.head()

# 二、模型训练

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import numpy as np
print(tf.__version__)

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
import tokenization

In [ ]:
EPOCHS=3

## 编码和建模函数

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    print(texts)
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)


def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(3, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

## 加载模型与训练

In [ ]:
#load BERT as a KerasLayer
%time
#module_url = "https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/1"
module_url="https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
tokenizer.tokenize("学习自然语言处理的BERT模型")

In [ ]:
train_input = bert_encode(train['content'].astype(str), tokenizer, max_len=200)
test_input = bert_encode(test['content'].astype(str), tokenizer, max_len=200)

In [ ]:
train_labels = train['target'].astype(int)+1
train_labels

In [ ]:
model = build_model(bert_layer, max_len=200)
model.summary()

In [ ]:
train_history = model.fit(
    train_input, train_labels,
    epochs=EPOCHS,
    validation_split=0.2,
    batch_size=32
)

In [ ]:
model.save_weights('bert_model_epoch3.h5')

In [ ]:
test_pred = model.predict(test_input)
predictions = np.argmax(test_pred, axis=-1)-1
print(predictions)

In [ ]:
predictions

In [ ]:
predictions=pd.DataFrame(predictions)
predictions.to_csv('submission.csv', index=False)